In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.models import Model, load_model
from keras.layers import LSTM, Dense, TimeDistributed, Input, Masking, RepeatVector, Bidirectional, Embedding
nltk.download('punkt')
import string
punc = list(string.punctuation)
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
print(stop_words)

Using TensorFlow backend.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sourabhbhattacharjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.metrics import pairwise
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle
import csv
import os
from scipy.spatial.distance import cosine
import time

In [33]:
## HYPER-PARAMETERS

num_words = 10000
MAX_SEQUENCE_LENGTH = 16
WORD_EMBEDDING_DIM = 200
batch_size = 640
ENCODING_DIM = 500
NUM_DATAPOINT = -1 # All in the file
NUM_ICD_CODES = 6553
NUM_EPOCHS = 2
VALIDATION_SPLIT = 0.2
PATIENCE = 5
TOP_K_PER_SENTENCE = 10
similarity_threshold = 0.7

THRESHOLD = -1 # TODO

ICD_DESC_FILENAME = 'ICD_desc_with_freq.csv'
#DD_FILENAME = 'MIMIC_III_Final_1000_clean_shorted.csv'
DD_FILENAME = 'MIMIC-III-Final_5000_clean_shortened.csv'
#DD_FILENAME = 'MIMIC-III-Final_cleaned.csv'
EMB_FILE = 'PMC-w2v.bin'

#RESULT_FILENAME = 'Experiment Results.xlsx'
#http://yaronvazana.com/2019/09/28/training-an-autoencoder-to-generate-text-embeddings/

In [20]:
#Calculate Sentence Embedding using Average Word Embedding of tokens
def get_sentence_embedding(sent, embedding):
    words = word_tokenize(sent)
    m = len(words)
    total_embedding = 0
    
    for i in range(m):
        try:
            w = embedding[words[i]]
        except:
            w = embedding['unk']
        
        total_embedding += w
    
    return total_embedding/float(m)

In [5]:
#Load Word Embedding (Pub Med W2V)
from gensim.models.keyedvectors import KeyedVectors
word_model = KeyedVectors.load_word2vec_format(EMB_FILE, binary=True)
print('Found %s word vectors.' % len(word_model.index2word))
print(word_model['Cholera'].shape)

Found 2515686 word vectors.
(200,)


# ICD File

In [34]:
ICD_desc_filename = ICD_DESC_FILENAME
df_icd = pd.read_csv(ICD_desc_filename, encoding = 'latin1')

df_icd = df_icd[['Code', 'Long Description']]

print(df_icd.shape)

print('\n', df_icd.head())

(6553, 2)

     Code                                   Long Description
0   4019                 Unspecified essential hypertension
1   4280              Congestive heart failure, unspecified
2  42731                                Atrial fibrillation
3  41401  Coronary atherosclerosis of native coronary ar...
4   5849                  Acute kidney failure, unspecified


In [35]:
df_icd_cleaned = df_icd
df_icd_cleaned['Sentence Embedding'] = ''

for i in range(df_icd.shape[0]):
    sent = df_icd_cleaned['Long Description'][i]
    sent_tokenized = word_tokenize(sent)
    
    clean_sent = ''
    
    for token in sent_tokenized:
        if token.lower() not in punc and token.lower() not in stop_words:
            clean_sent = clean_sent + ' ' + token.lower()
    
    df_icd_cleaned['Long Description'][i] = clean_sent
    
    df_icd_cleaned['Sentence Embedding'][i] = get_sentence_embedding(df_icd_cleaned['Long Description'][i], word_model)
    
df_icd_cleaned.head()    

,Code,Long Description,Sentence Embedding
0,4019,unspecified essential hypertension,"[-0.19575113, 0.050484493, 0.09391897, 0.07607..."
1,4280,congestive heart failure unspecified,"[-0.38989657, 0.20555413, 0.004459345, 0.05428..."
2,42731,atrial fibrillation,"[-0.05242303, 0.11609671, -0.13533235, 0.17891..."
3,41401,coronary atherosclerosis native coronary artery,"[-0.5348693, -0.063154265, 0.26152247, 0.36165..."
4,5849,acute kidney failure unspecified,"[-0.39448828, 0.24474034, -0.03692953, -0.0219..."


In [36]:
#Capture the ICD embedding as a numpy matrix
icd_sent_emb = df_icd_cleaned['Sentence Embedding'][:]
icd_sent_emb_list = [x for x in icd_sent_emb]

icd_sent_emb_matrix = np.asarray(icd_sent_emb_list, dtype=np.float32)
icd_sent_emb_matrix.shape

(6553, 200)

In [37]:
icd_sentences = list(df_icd_cleaned.head(NUM_ICD_CODES)['Long Description']) # Number of sentences
icd_codes = list(df_icd_cleaned.head(NUM_ICD_CODES)['Code'])
icd_index = list(range(NUM_ICD_CODES))

print(len(icd_sentences), len(icd_codes), len(icd_index))
print("ICD_Sentences = ", len(icd_sentences))

6553 6553 6553
ICD_Sentences =  6553


In [38]:
# 3 Dictionary of ICD index, ICD Code and ICD desc

# index, codes
dict_index_code = dict(zip(icd_index, icd_codes))

# codes, index
dict_code_index = dict(zip(icd_codes, icd_index))

#index, code desc
dict_index_desc = dict(zip(icd_index, icd_sentences))

#icd sentence embedding matrix slice
icd_sent_emb_matrix = icd_sent_emb_matrix[:NUM_ICD_CODES]

# Cleaned MIMIC-III DD file

In [39]:
DD_filename = DD_FILENAME
df_DD = pd.read_csv(DD_filename)
FILESIZE = df_DD.shape[0]
df_DD.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,clean_text,top_ICD
0,16260,{Admission Date: [**2180-2-5**] ...,130750.0,"['0389', '51881', '5070', '4280', '4820', '054...",admission date discharge date recorded as havi...,"['51881', '5070', '4280', '4820', '70703', '43..."
1,41507,{Admission Date: [**2173-12-17**] ...,178712.0,"['99671', '42832', '07030', '4280', '2724', '0...",admission date discharge date of birth sex nam...,"['99671', '42832', '4280', '2724', 'V4365', '4..."
2,46401,{Admission Date: [**2197-9-11**] Discharg...,188022.0,"['431', '3314', '51881', '2720', '4019']",admission date discharge date of present illne...,"['431', '3314', '51881', '2720', '4019']"
3,45157,{Admission Date: [**2167-5-19**] ...,185673.0,"['4210', '5856', '7907', '5070', '25013', '261...",admission date discharge date of birth sex nam...,"['4210', '5856', '7907', '5070', '25013', '261..."
4,45351,{Admission Date: [**2153-12-27**] ...,186027.0,"['80021', '5559', '80121', '81000', 'E8889', '...",admission date discharge date of birth sex kno...,"['80021', '5559', '80121', '81000', 'E8889', '..."


In [40]:
DD_Data = df_DD[['ICD9_CODE', 'clean_text']]
DD_Data

,ICD9_CODE,clean_text
0,"['0389', '51881', '5070', '4280', '4820', '054...",admission date discharge date recorded as havi...
1,"['99671', '42832', '07030', '4280', '2724', '0...",admission date discharge date of birth sex nam...
2,"['431', '3314', '51881', '2720', '4019']",admission date discharge date of present illne...
3,"['4210', '5856', '7907', '5070', '25013', '261...",admission date discharge date of birth sex nam...
4,"['80021', '5559', '80121', '81000', 'E8889', '...",admission date discharge date of birth sex kno...
...,...,...
4995,"['V3001', '7742', '76518', '76528', '7783', 'V...",admission date discharge date of birth sex bab...
4996,"['03842', '5990', '5845', '70703', '42731', '2...",admission date discharge date of birth sex sul...
4997,"['1913', '78039', '3488', '53081', '4430', '36...",admission date discharge date of birth sex rec...
4998,"['29181', '30560', '30391', '07054', '2752', '...",admission date discharge date of birth sex nam...


In [41]:
#Calculate Cosine similarity between 2 word vectors
def cosine_sim_matrix(sent_emb, icd_emb = icd_sent_emb_matrix):
    similarity_matrix = np.dot(sent_emb, icd_emb.T)/(np.linalg.norm(sent_emb)*np.linalg.norm(icd_emb, axis = 1))
    return similarity_matrix #num/denom

In [42]:
#Convert ICD Indices to hot vectors of length: NUM_ICD_CODE
def generate_binary(icd_index_list, total_icd = NUM_ICD_CODES):
    binary_icd = np.zeros(total_icd)
    for x in icd_index_list:
        binary_icd[x] = 1
        
    return binary_icd

In [43]:
#Calculate Specificity

def calc_specificity(pred_vec, act_vec):
    m = len(pred_vec)
    
    specificity = []
    
    for i in range(m):
        pred, act = pred_vec[i], act_vec[i]
        dict_XY = {'pred':pred, 'act':act}
        res = pd.DataFrame(dict_XY, columns=['pred', 'act'])
        res['Sum'] = res['pred'] + res['act']
        
        pred_p = res[res.pred == 1]
        pred_n = res[res.act == 1]
        
        tp = len(res[res.Sum == 2])
        tn = len(res[res.Sum == 0]) 
        
        fp = len(pred_p[pred_p.act == 0])
        fn = len(pred_n[pred_n.act == 1])
        
        #Calculate specificity for this example:
        spec = tn/(tn + fp)
        
        specificity.append(spec)
    
    return sum(specificity) / float(m)
        

In [44]:
#Predict ICD 9 Codes

#Initialize Results Dataframe which will store the relevant metrics for each threshold
results = pd.DataFrame(columns = ['Number of Observations', 
                                  'Number of ICD Codes',
                                  'Threshold',
                                  'Precision', 
                                  'Recall', 
                                  'F1-Score', 
                                  'Specificity'])

threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#threshold_list = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for thresh in threshold_list:

    #Print Threshold:
    print('\nCommenced for Similarity Threshold:', thresh)
    
    start_time = time.time()
    
    X = DD_Data['clean_text']
    Y = list(DD_Data['ICD9_CODE'])

    predicted_codes_index = []
    actual_codes_index = []

    m = len(Y)

    for i in range(m):
        text = X[i]

        #Convert Actual Codes to indices. Consider only those codes present in icd_codes
        act_icd_indices = [dict_code_index[y] for y in Y[i][2:-2].split("', '") if y in icd_codes] #To get rid of '' and ,

        #Initialize Prediction list. This will contain indices of similar icd codes
        pred = []


        sent_emb = get_sentence_embedding(text, word_model)

        #Calculate cosine similarity of sentence with icd sentences
        sim = cosine_sim_matrix(sent_emb)

        #Apply thresholding to obtain indices of similar embeddings
        threshold_filter = sim >= thresh

        #Similarity >= threshold --> similarity 
        #Similarity < threshold --> 0
        sim_post_threshold = sim*threshold_filter

        #Add indices of non zero elements to pred
        pred.extend(np.nonzero(sim_post_threshold)[0].tolist())

        #Consider the unique indices
        pred = set(pred)

        #Convert list of indices to hot vector
        pred_icd_vector = generate_binary(pred)
        act_icd_vector = generate_binary(act_icd_indices)

        #Append
        predicted_codes_index.append(pred_icd_vector)
        actual_codes_index.append(act_icd_vector)


        if i%1000 == 0 and i != 0:
            print(i, ' sentences done.')

    #Calculate Precision, Recall, F1 Score for this threshold
    precision, recall, f1_score, support = precision_recall_fscore_support(actual_codes_index, predicted_codes_index, average = 'micro')
    specificity = calc_specificity(predicted_codes_index, actual_codes_index)

    #Add to results Dataframe:
    results = results.append({'Number of Observations': m, 
                    'Number of ICD Codes': NUM_ICD_CODES, 
                    'Threshold': thresh,
                    'Precision': precision, 
                    'Recall': recall, 
                    'F1-Score': f1_score, 
                    'Specificity': specificity}, ignore_index=True)

    print('\nPrecision: ', precision, ' Recall: ', recall, ' F1-Score: ', f1_score, ' Specificity:', specificity)
    
    time_elapsed = time.time() - start_time
    print('\nTime elapsed: ', time_elapsed/60.0, ' minutes')

    
#Print Results
RESULT_FILENAME = 'Results/Experiment_Results_Documentwise_' + str(DD_Data.shape[0]) + '_records_' + str(NUM_ICD_CODES) + '_codes_' + '.xlsx'
results.to_excel(RESULT_FILENAME) 
print('Results Saved!')
results


Commenced for Similarity Threshold: 0.1
1000  sentences done.
2000  sentences done.
3000  sentences done.
4000  sentences done.

Precision:  0.001695187884142253  Recall:  0.998662038041513  F1-Score:  0.0033846304967193735  Specificity: 0.0055666357341900615

Time elapsed:  2.637507351239522  minutes

Commenced for Similarity Threshold: 0.2
1000  sentences done.
2000  sentences done.
3000  sentences done.
4000  sentences done.

Precision:  0.0017335807127825025  Recall:  0.9815216605192739  F1-Score:  0.0034610484612771857  Specificity: 0.044312030656941934

Time elapsed:  1.9266691525777182  minutes

Commenced for Similarity Threshold: 0.3
1000  sentences done.
2000  sentences done.
3000  sentences done.
4000  sentences done.

Precision:  0.0018216041985799562  Recall:  0.8713386851811673  F1-Score:  0.0036356078640238  Specificity: 0.19265295865030088

Time elapsed:  2.0111791133880614  minutes

Commenced for Similarity Threshold: 0.4
1000  sentences done.
2000  sentences done.
300

/Users/sourabhbhattacharjee/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Precision:  0.0  Recall:  0.0  F1-Score:  0.0  Specificity: 1.0

Time elapsed:  2.6035815954208372  minutes
Results Saved!


,Number of Observations,Number of ICD Codes,Threshold,Precision,Recall,F1-Score,Specificity
0,5000.0,6553.0,0.1,0.001695,0.998662,0.003385,0.005567
1,5000.0,6553.0,0.2,0.001734,0.981522,0.003461,0.044312
2,5000.0,6553.0,0.3,0.001822,0.871339,0.003636,0.192653
3,5000.0,6553.0,0.4,0.001985,0.576300,0.003955,0.509942
4,5000.0,6553.0,0.5,0.001832,0.192106,0.003629,0.822994
5,5000.0,6553.0,0.6,0.001504,0.036016,0.002888,0.959573
6,5000.0,6553.0,0.7,0.002347,0.006979,0.003513,0.994984
7,5000.0,6553.0,0.8,0.024768,0.000145,0.000288,0.999990
8,5000.0,6553.0,0.9,0.000000,0.000000,0.000000,1.000000
